# Etapa 1: Instalação de bibliotecas

In [1]:
!pip install pandas pandas_datareader tqdm

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
%config Completer.use_jedi = False

# Etapa 2: Import das bibliotecas

In [3]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import  matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [4]:
tf.__version__

'2.5.0'

# Etapa 3: Construção da IA para negociação de ações

In [5]:
class AI_Trader():
    
    def __init__(self, state_size, action_space = 3, model_name = "AITrader"):
        """
            state_size: tamanho da camada de entrada, que informa quantos dados 
                        estão chegando sobre o estado do ambiente/problema.
                        
            action_space: número de ações que podem ser realizadas.
            
            memory: número de ações que podem ser armazenadas na memória dentro
                    do batch que será usado para o treinamento.
                    
            gamma: fator de desconto na função de Bellman.
            
            epsilon: percentual de vezes que será uma ação randomica e não da rede neural.
            
            epsilon_final: percentual mínimo de ações tomadas randomicamente, que são ações
                           que devem continuar a existir para que o agente possa aprender
                           novas ações.
            
            epsilon_decay: vai decrementar e epsilon para que as ações comecem a ser decididas
                           ela rede neural.
        """
        
        self.state_size = state_size
        self.action_space = action_space
        self.memory = deque(maxlen = 2000)
        self.model_name = model_name
        
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995
        self.model = self.model_builder()
    
    def model_builder(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=self.state_size))
        model.add(tf.keras.layers.Dense(units=64, activation="relu"))
        model.add(tf.keras.layers.Dense(units=128, activation="relu"))
        model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
        return model
    
    def trade(self, state):
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)
        
        actions = self.model.predict(state[0])
        return np.argmax(actions[0])
    
    def batch_train(self, batch_size):
        batch = []
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])
        
        for state, action, reward, next_state, done in batch:
            if not done:
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            
            target = self.model.predict(state)
            target[0][action] = reward
            
            self.model.fit(state, target, epochs=1, verbose=0)
        
        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay

# Etapa 4: Pré-processamento da base de dados

## Funções auxiliares

### Sigmoid

Usar como normalização, para manter os preços em 0 e 1.

In [6]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [7]:
sigmoid(0.5)

0.6224593312018546

### Formatação de preços

In [8]:
def stock_price_format(n):
    if n < 0:
        return "- $ {0:2f}".format(abs(n))
    else:
        return "$ {0:2f}".format(abs(n))

In [9]:
stock_price_format(-10)

'- $ 10.000000'

### Carregador da base de dados

In [10]:
dataset = pd.read_csv('AAPL.csv', index_col='Date', dayfirst=True)
dataset.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,7.622500,7.660714,7.585000,7.643214,6.572422,493729600
2010-01-05,7.664286,7.699643,7.616071,7.656429,6.583784,601904800
2010-01-06,7.656429,7.686786,7.526786,7.534643,6.479061,552160000
2010-01-07,7.562500,7.571429,7.466071,7.520714,6.467082,477131200
2010-01-08,7.510714,7.571429,7.466429,7.570714,6.510078,447610800


In [11]:
def dataset_loader(stock_name):
    dataset = pd.read_csv(stock_name, index_col='Date', dayfirst=True)
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]
    close = dataset['Close']
    return close

## Criados de estados

In [12]:
def state_creator(data, timestep, window_size):
    starting_id = timestep - window_size + 1
    
    if starting_id >= 0:
        windowed_data = data[starting_id:timestep + 1]
    else:
        windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])
    
    state = []
    for i in range(window_size - 1):
        state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
    return np.array([state]), windowed_data

### Carregamento da base de dados

In [13]:
data = dataset_loader('AAPL.csv')

In [14]:
s, w = state_creator(data, 1, 5)

# Etapa 5: Treinando a IA

## Configuração dos hyper parâmetros

In [15]:
window_size = 10
episodes = 1000 # são como épocas
batch_size = 32
data_samples = len(data) - 1

## Definição do modelo

In [16]:
trader = AI_Trader(window_size)

In [17]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


## Loop de treinamento

In [18]:
for episode in range(1, episodes + 1):
    print('Episode: {}/{}'.format(episode, episodes))
    state = state_creator(data, 0, window_size + 1)
    total_profit = 0
    trader.inventory = []
    for t in tqdm(range(data_samples)):
        action = trader.trade(state)
        next_state = state_creator(data, t + 1, window_size + 1)
        reward = 0
        
        if action == 1: # Comprando uma ação
            trader.inventory.append(data[t])
            print("AI Trader bought: ", stock_price_format(data[t]))
        elif action == 2 and len(trader.inventory) > 0: # Vendendo uma ação
            buy_price = trader.inventory.pop(0)
            
            reward = max(data[t] - buy_price, 0)
            total_profit += data[t] - buy_price
            print("AI Trader sold: ", stock_price_format(data[t]), " Profit: ", stock_price_format(data[t] - buy_price))
            
            if t == data_samples - 1:
                done = True
            else:
                done = False
            
            trader.memory.append((state[0], action, reward, next_state[0], done))
            
            state = next_state
            
            if done:
                print("###############################")
                print("Total profit: {}".format(total_profit))
                print("###############################")
            
            if len(trader.memory) > batch_size:
                trader.batch_train(batch_size)
        
        if episode % 10 == 0:
            trader.model.save("ai_trader_{}.h5".format(episode))

  0%|          | 0/2894 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 7.523214
AI Trader bought:  $ 7.479643
AI Trader bought:  $ 7.354643
AI Trader sold:  $ 7.680000  Profit:  $ 0.156786
AI Trader bought:  $ 7.561786
AI Trader bought:  $ 7.431071
AI Trader bought:  $ 7.062500
AI Trader bought:  $ 7.355000
AI Trader sold:  $ 6.995000  Profit:  - $ 0.484643
AI Trader bought:  $ 7.115357
AI Trader sold:  $ 6.858929  Profit:  - $ 0.495714
AI Trader sold:  $ 6.932857  Profit:  - $ 0.628929
AI Trader sold:  $ 7.006786  Profit:  - $ 0.424285
AI Trader bought:  $ 6.968571
AI Trader sold:  $ 7.095357  Profit:  $ 0.032857
AI Trader sold:  $ 7.156429  Profit:  - $ 0.198571
AI Trader bought:  $ 7.264286
AI Trader sold:  $ 7.247500  Profit:  $ 0.132143
AI Trader sold:  $ 7.202500  Profit:  $ 0.233929
AI Trader sold:  $ 7.157857  Profit:  - $ 0.106429
AI Trader bought:  $ 7.463929
AI Trader bought:  $ 7.458929
AI Trader sold:  $ 7.476071  Profit:  $ 0.012142
AI Trader sold:  $ 7.525357  Profit:  $ 0.066428
AI Trader bought:  $ 7.8

  4%|▍         | 110/2894 [00:02<01:12, 38.17it/s]

AI Trader sold:  $ 9.081429  Profit:  $ 0.395715


  4%|▍         | 114/2894 [00:05<02:27, 18.79it/s]

...

AI Trader bought:  $ 27.497499
AI Trader sold:  $ 27.780001  Profit:  $ 0.282502


 58%|█████▊    | 1685/2894 [02:34<00:24, 49.62it/s]

AI Trader bought:  $ 28.395000
AI Trader sold:  $ 28.392500  Profit:  - $ 0.002500


 60%|██████    | 1742/2894 [02:38<01:44, 11.00it/s]


KeyboardInterrupt: 